# Run SuperPrompt-v1 AI Model NO UI

Make your prompts better for AI Art or in general!

Used Model: https://huggingface.co/roborovski/superprompt-v1

Google Colab Notebook Made by [Nick088](https://linktr.ee/Nick088)

In [ ]:
#@title Install & Load Dependencies, Model

#@markdown If you wanna use CPU (slower, no daily limit): Set the CPU from Edit -> Notebook Settings -> CPU

#@markdown If you wanna use GPU (faster, max 12 free hours daily limit): Set the Video Card from Edit -> Notebook Settings -> T4 GPU OR Any other GPUs based on your Google Colab Subscription

#@markdown Anyways its a very small model, it doesn't matter much if you use cpu or gpu.

!pip install transformers
!pip install einops
!pip install accelerate
!pip install sentencepiece
import torch
from IPython.display import clear_output
from transformers import T5Tokenizer, T5ForConditionalGeneration
import random

if torch.cuda.is_available():
    device = "cuda"
    print("Using GPU")
else:
    device = "cpu"
    print("Using CPU")

if Precision_Model_Type == 'fp32':
  Precision_Model_Type = 'torch.float32'
else:
  Precision_Model_Type = 'torch.float16'

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
model = T5ForConditionalGeneration.from_pretrained("roborovski/superprompt-v1", torch_dtype=torch.float16) # torch.float16 is basically fp16, so model precision in 16 bits which is faster and less resource consuming, you could also put torch.float32 which is fp32 that lods it in 32bits which is more precise but slower and more resource consuming

model.to(device)

clear_output()
print(f"Downloaded & Loaded SuperPrompt-v1 on {'GPU' if device == 'cuda' else 'CPU'}")

In [ ]:
#@title Run Inference
#@markdown Write here the prompt you want to upsample to have better detailed description.

input_text = "Expand the following prompt to add more detail: A storefront with 'Text to Image' written on it." #@param {type:"string"}
#@markdown Maximum number of the tokens to generate, controls how long is the text.
max_new_tokens = 512 #@param {type:"slider", min:250, max:512, step:1}
#@markdown Penalize repeated tokens, so make the AI repeat less of itself
repetition_penalty = 1.2 #@param {type:"slider", min:0.0, max:2.0, step:0.05}
#@markdown Higher values produce more diverse outputs
temperature = 0.5 #@param {type:"slider", min:0.0, max:1.00, step:0.05}
#@markdown Higher values sample more low-probability tokens
top_p = 1 #@param {type:"slider", min:0.0, max:2.0, step:0.05}
#@markdown Higher k means more diverse outputs by considering a range of tokens
top_k = 1 #@param {type:"slider", min:1, max:100, step:1}
#@markdown A starting point to initiate the generation process, put 0 for random seed
seed = 42 #@param {type:"integer"}

if seed == 0:
    seed = random.randint(1, 100000)
    torch.manual_seed(seed)
else:
    torch.manual_seed(seed)

input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
outputs = model.generate(input_ids, max_new_tokens=max_new_tokens, repetition_penalty=repetition_penalty, do_sample=True, temperature=temperature, top_p=top_p, top_k=top_k)

dirty_text = tokenizer.decode(outputs[0])
text = dirty_text.replace("<pad>", "").replace("</s>", "")
print(text)